Paillier Cipher Demo
==================================

This notebook has been created to provide a basic walkthrough of the crypto scheme invented by P. Paillier in 1999.

__Note for using Jupyter:__ Use $\texttt{SHIFT + ENTER}$ to execute the code snippets!

We start by recalling the problem on which the system is based: the __Decisional Composite Residuosity Problem (DCRP)__. Recall the __DCRP hardness assumption__:

Given a security parameter $\lambda = 1^n$ and PPT algorithm $\texttt{GenModulus}$ the DCRP is hard relative to $\texttt{GenModulus}$, if every PPT adversary $\mathcal{A}$ has negligible advantage to solve it:

$$
    \texttt{Adv}^{\text{dcrp}}_{\mathcal{A}, \texttt{GenModulus}}(n) = 
    \mathbb{P}[\mathcal{A}(N, r^N \text{ mod } N^2) = 1] - 
    \mathbb{P}[\mathcal{A}(N, r) = 1] \leq \text{negl}(n),
$$

where $(N, p, q) \leftarrow \texttt{GenModulus(n)}$ and $r$ is picked uniformly from $\mathbb{Z}_{N^2}^*$.

Informally, the DCRP assumption says that for a fitting $\texttt{GenModulus}$, telling apart a generic group element from an $N$-th power is very hard (NP-hard in fact).

In [1]:
load('paillier.sage')

Key Generation
---------------------------------------------------------------

Key generation is quite straightforward: we simply need to find a fitting $N$ for the public key, and the corresponding $\phi(N)$ for the secret key.

Thus, the algorithm is as follows:

1. Given security parameter $1^n$, pick two different random primes $p, q$ such that $|p| = |q|$. (This is the implementation of $\texttt{GenModulus}$.)

2. Calculate $N = pq$ and $\phi(N) = (p - 1)(q - 1)$. Since $\gcd(N, \phi(N)) = 1$, $\phi(N)$ can be inverted mod $N$, and so we calculate this as well.

3. Output $(pk, sk)$, where $pk = N$ and $sk = (N, \phi(N), \phi(N)^{-1})$.

In [2]:
pk, sk = key_gen(50);pk, sk

(680870048931612002542069545043,
 (680870048931612002542069545043,
  680870048931610329172080889000,
  18335112612710301625254405773))

Encryption
--------------------------------------------------------------

To leverage the DCRP assumption, we would like to operate in the group $\mathbb{Z}_{N^2}^*$ and encode our message in its elements. To do this, we will leverage the isomorphism between $\mathbb{Z}_{N^2} \cong \mathbb{Z}_N \times \mathbb{Z}_N^*$ given by

$$
    (1 + N)^a \cdot b^N \text{ mod } N^2 \cong (a, b) \text{ mod } N
$$

In particular, we note that for some message $m \in \mathbb{Z}_N$ and a random element $r \in \mathbb{Z}_N^*$, we can have

$$
    (m, 0) \cdot (1, r) = (m, r).
$$

Then, we can really simply calculate the image of the isomorphism for $(m, r)$. This is precisely what we perform in the encryption function:

In [3]:
m1 = 1000
m2 = 1111

c1 = encrypt(pk, m1)
c2 = encrypt(pk, m2)

c1, c2

(317242678010904977962638791784124390822053092865154142987794,
 183547091357144087819490270157715711762618451355354599492903)

Decryption
-------------------------------------------------------------------

Given some secret key $sk = (N, \phi(N), \phi(N)^{-1})$ and, the decryption function will calculate and output

$$
    m' = \frac{(c^{\phi(N)} \text{ mod } N) - 1}{N} \cdot \phi(N)^{-1} \text{ mod } N.
$$

Correctness
--------------------------------------------------------------------

The above might be quite a mouthful, but thinking in the context of the isomorphism, it is, in fact, a very natural set of calculations. Proving correctness takes some care, please see Section 7.2.2 in the report for a proof.

Here, we check the correctness empirically:

In [4]:
m1_p = decrypt(sk, c1)
m2_p = decrypt(sk, c2)

m1_p, m2_p

(1000, 1111)

Homomorphic Property
------------------------------------------------------------

For the two ciphertexts $c_1 \cong (m_1, r_1), c_2 \cong (m_2, r_2)$, observe that multiplying them together we get

$$
    c_1 \cdot c_2 \text{ mod } N^2\cong (m_1, r_1) \cdot (m_2, r_2)  \text{ mod } N = (m_1 + m_2, r_1r_2) \text{ mod } N
$$

which is again a valid ciphertext. Thus multiplication of ciphertexts (mod $N^2$) corresponds to the addition of their underlying plaintexts (mod $N$)! 

In [5]:
decrypt(sk, c1 * c2)

2111

Security Considerations
-------------------------------------------------------

We have seen in Section 7.1.2 that no homomorphic scheme can be secure in the non-malleability framework. In particular, we can expose a PPT adversary $\mathcal{A}$ that will break this notion:

In [12]:
def A(pk, challenge=None):
    
    N = pk
    
    # If we don't pass a challenge, we are in stage 1
    if challenge is None:
        
        # The sampling function we will return will
        # uniformly sample M\{1} where M is the message space group
        # Note that the identity in Z_N is 0!
        # Note that for efficiency, we sample at most the first 99 elements
        sample = lambda: ZZ.random_element(1, min(N, 100))
        
        return sample
    
    else:
        
        # We calculate the square of the received ciphertext
        c = challenge * challenge
        
        # Also, define the relation R(a, b) iff a^2 = b
        R = lambda a, b: a * a == b
        
        return c, R

Now, we define the NM-CPA experiment:

In [13]:
def Expr(sec_param, Pi, A):
    """
    NM-CPA security experiment.
    
    We take a security parameter lambda, 
    a cipher Pi and PPT adversary A
    """
    (gen, enc, dec) = Pi
    
    # Step 1: generate key pair
    (pk, sk) = gen(sec_param)
    
    # Step 2: obtain sampling algorithm from the adversary
    samp = A(pk)
    
    # Step 3: generate a message using the adversary's sampling
    # algorithm and calculate the challenge
    m = samp()
    
    challenge = enc(pk, m)
    
    # Step 4: see if the adversary succeeds
    c, R = A(pk, challenge=challenge)
    
    return 1 if R(challenge, c) else 0

Now, we will run 100 experiments:

In [14]:
Pi = (key_gen, encrypt, decrypt)

sec_param = 20

counter = 0;

for i in range(100):
    # Count the number of times the adversary solves the experiment
    counter += Expr(sec_param, Pi, A)
    
counter

100

We see that $\mathcal{A}$ breaks NM-CPA security with probability 1, which is a non-negligible advantage, hence Paillier is not NM-CPA secure.